In [1]:
##this is to test the efficacy of medusa over the internal cobra gapfiller


import cobra
from cobra import Metabolite, Gene, Reaction
import pandas as pd
import medusa
from medusa.test import load_universal_modelseed
import matplotlib.pylab as plt
import pandas as pd


In [2]:
# loop through the sheets of the spreadsheet; each sheet is a biological replicate
# of all 3 of the biolog plates. Columns are time in hours.
reps = {}
for i in range(0,4):
    biolog_raw = pd.read_excel('../data/experiments/biolog_raw.xlsx',sheet_name="rep_"+str(i+1),index_col=0)
    biolog_raw.columns = [col for col in biolog_raw.columns]
    reps[i] = biolog_raw
    
all_biolog = pd.concat([reps[0],reps[1],reps[2],reps[3]])

# convert to mean and std at each time point, 
# and get the max OD at all time points (from means)
mets = reps[0].index.unique()
means = {}
stds = {}
maxODs = {}
for met in mets:
    vals = pd.DataFrame()
    concat_axis = 1
    if met == 'Neg':
        concat_axis = 0
    
    for rep in reps:
        vals = pd.concat([vals,reps[rep].loc[met]],axis=concat_axis)
    
    if met != 'Neg':
        vals = vals.T
    
    means[met] = vals.mean()
    stds[met] = vals.std()
    maxODs[met] = means[met].max()

# Write the condensed biolog data to a spreadsheet
all_biolog.to_csv("../data/all_biolog.csv")

In [3]:
# threshold the growth data to get positive growth conditions only
growth_threshold = 0.3
positive_growth_conditions = {condition:maxODs[condition] for condition in maxODs.keys() if maxODs[condition] > growth_threshold}
# Drop inulin, which is a polymer that cannot be represented in the modelseed namespace
positive_growth_conditions.pop('Inulin')
phenotypes = pd.DataFrame.from_dict(positive_growth_conditions,orient='index')
phenotypes.columns = ["Max OD"]

# Read in the bioog:seed file and merge with the positive growth data to get SEED compound ids for each metabolite
biolog_to_seed = pd.read_csv('../data/biolog_names_to_seed.tsv', sep = '\t')
biolog_to_seed.index = biolog_to_seed["name"]
biolog_to_seed = biolog_to_seed.drop("name", axis=1)
carbon_sources = phenotypes.index.tolist()
phenotypes = phenotypes.merge(biolog_to_seed, left_index=True, right_index=True)
carbon_sources_post_merge = phenotypes.index.tolist()
# Print out the metabolites missing identifiers. These might be expected (e.g. pectin is a polymer that
# cannot be properly represented, so we don't include it)
print("The following compounds are missing; is this expected?", set(carbon_sources) - set(carbon_sources_post_merge))

The following compounds are missing; is this expected? {'Pectin'}


In [4]:
# Load the biolog data from Plata et al., Nature 2014
from medusa.test import load_biolog_plata
biolog_base_composition, biolog_base_dict, biolog_thresholded = load_biolog_plata()

# Remove Heme and H2S2O3 from the biolog base--these compounds are not actually in the base, but are frequently 
# required by modelseed models.
biolog_base_composition = biolog_base_composition.drop(biolog_base_composition.index[biolog_base_composition['ID'] == 'cpd00028_e'])
biolog_base_composition = biolog_base_composition.drop(biolog_base_composition.index[biolog_base_composition['ID'] == 'cpd00268_e'])
if "cpd00028_e" in biolog_base_dict:
    del biolog_base_dict["cpd00028_e"]
if "cpd00268_e" in biolog_base_dict:
    del biolog_base_dict["cpd00268_e"]
biolog_base_composition

,Name,ID
0,H2O,cpd00001_e
1,O2,cpd00007_e
2,Phosphate,cpd00009_e
3,CO2,cpd00011_e
4,NH3,cpd00013_e
5,Mn2+,cpd00030_e
6,Zn2+,cpd00034_e
7,Sulfate,cpd00048_e
8,Cu2+,cpd00058_e
9,Ca2+,cpd00063_e


In [5]:
# load the universal reaction database
from medusa.test import load_universal_modelseed
from cobra.core import Reaction
import cobra

# load the modelseed universal model. This will take a few minutes.
universal = load_universal_modelseed()

# load the psuedomonas syringae draft recon
seed_draft = cobra.io.load_json_model('../data/modelseed_data/modelseed_draft_psy_DC3000.json')

# add all reactions from the draft to the universal if they are not already present
add_to_universal = []
for rxn in seed_draft.reactions:
    if rxn.id != 'bio1':
        if rxn.id not in [r.id for r in universal.reactions]:
            add_to_universal.append(rxn.copy())

universal.add_reactions(add_to_universal)

In [6]:
##for gapfilling purposes, also load in the universal seed model to pool with reactions from model
##use the most recent version of the model (currently should be version 8)
model = cobra.io.read_sbml_model("../results/reconstructions/pstv8.xml")
model.objective = 'bio1'
model

No objective coefficients in model. Unclear what should be optimized


Name,PST
Memory address,0x0124f76898
Number of metabolites,1174
Number of reactions,1161
Number of groups,58
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"c, e"


In [7]:
add_mets = []
add_exchanges = []
for met in list(biolog_base_dict.keys()):
    try:
        model.metabolites.get_by_id(met)
    except:
        print('no '+met)
        add_met = universal.metabolites.get_by_id(met).copy()
        add_mets.append(add_met)

for met in list(biolog_base_dict.keys()):
    # Search for exchange reactions
    try:
        model.reactions.get_by_id('EX_'+met)
    except:
        add_met = universal.metabolites.get_by_id(met)
        ex_rxn = Reaction('EX_' + met)
        ex_rxn.name = "Exchange reaction for " + met
        ex_rxn.lower_bound = -1000
        ex_rxn.upper_bound = 1000
        ex_rxn.add_metabolites({add_met:-1})
        add_exchanges.append(ex_rxn)
        if add_met.id not in [m.id for m in add_mets]:
            add_mets.append(add_met)

no cpd00001_e
no cpd00034_e
no cpd00099_e


In [8]:
# Find metabolites from the biolog data that are missing in the test model
# and add them from the universal
missing_mets = []
missing_exchanges = []
media_dicts = {}

# remove duplicate conditions by using a set
for met_id in set(phenotypes['seed_id'].tolist()):
    media_dict = biolog_base_dict.copy()
    
    for single_met_id in met_id.split(','):
        # make a boolean indicator for metabolites missing from both
        # the model and the universal model
        missing = False
        
        # add the _e suffix for extracellular metabolites
        single_met_id = single_met_id + '_e'
        
        # search for the metabolite in the model.
        # If missing, try to find it in the universal.
        if single_met_id in [m.id for m in model.metabolites]:
            met = model.metabolites.get_by_id(single_met_id)
        else:
            if single_met_id in [m.id for m in universal.metabolites]:
                met = universal.metabolites.get_by_id(single_met_id)
            else:
                print(single_met_id + ' not in universal. Ignoring metabolite.')
                missing = True
        
        # If the metabolite was in the universal or the model,
        # check for an existing exchange reaction. If not there,
        # create and add the exchange reaction.
        if not missing:
            if 'EX_' + single_met_id not in [rxn.id for rxn in model.reactions]:
                ex_rxn = Reaction('EX_' + single_met_id)
                ex_rxn.name = "Exchange reaction for " + single_met_id
                ex_rxn.lower_bound = -1000
                ex_rxn.upper_bound = 1000
                ex_rxn.add_metabolites({met:-1})
                if ex_rxn.id not in [r.id for r in missing_exchanges]:
                    missing_exchanges.append(ex_rxn)
            if met_id in media_dicts.keys():
                # if media dict was already there, it means this is a double C/N
                # case (E.g. D+L mets). Don't need to alter the rest of the dict.
                media_dict['EX_'+single_met_id] = 1000
            else:
                media_dict = {'EX_'+k:v for k,v in media_dict.items()}
                media_dict['EX_'+single_met_id] = 1000
            
            media_dicts[met_id] = media_dict
            print(met_id + ' was not missing')


cpd00224 was not missing
cpd00280 was not missing
cpd00666 was not missing
cpd00129 was not missing
cpd00249 was not missing
cpd00027 was not missing
cpd00023 was not missing
cpd00082 was not missing
cpd00100 was not missing
cpd00246 was not missing
cpd00020 was not missing
cpd01293 was not missing
cpd00138 was not missing
cpd00130,cpd00386 was not missing
cpd00130,cpd00386 was not missing
cpd00106 was not missing
cpd00036 was not missing
cpd00105 was not missing
cpd01307 was not missing
cpd00108 was not missing
cpd00121 was not missing
cpd00609 was not missing
cpd02143_e not in universal. Ignoring metabolite.
cpd00154 was not missing
cpd00107 was not missing
cpd00222 was not missing
cpd00386 was not missing
cpd00053 was not missing
cpd00314 was not missing
cpd11585 was not missing
cpd00130 was not missing
cpd02351 was not missing
cpd00041 was not missing
cpd24420_e not in universal. Ignoring metabolite.
cpd00051 was not missing
cpd00076 was not missing
cpd00281 was not missing
cpd0011

In [9]:
# Add the exchange reactions for metabolites from biolog base
model.add_metabolites(add_mets)
print ("I have added" + str(add_mets))
model.add_reactions(add_exchanges)

# Do the same for single C/N supplements
model.add_metabolites(missing_mets)
model.add_reactions(missing_exchanges)

I have added[<Metabolite cpd00001_e at 0x12dff2358>, <Metabolite cpd00034_e at 0x12dff26a0>, <Metabolite cpd00099_e at 0x12dff2588>, <Metabolite cpd00007_e at 0x129bdc3c8>, <Metabolite cpd00009_e at 0x129253128>, <Metabolite cpd00011_e at 0x129d532b0>, <Metabolite cpd00013_e at 0x129521d68>, <Metabolite cpd00030_e at 0x1292d6be0>, <Metabolite cpd00048_e at 0x1295b39b0>, <Metabolite cpd00058_e at 0x12980b7f0>, <Metabolite cpd00063_e at 0x1299717f0>, <Metabolite cpd00067_e at 0x1293fbe80>, <Metabolite cpd00149_e at 0x12991fb38>, <Metabolite cpd00205_e at 0x125144470>, <Metabolite cpd00254_e at 0x129ac8668>, <Metabolite cpd00971_e at 0x129961780>, <Metabolite cpd10515_e at 0x129d12ef0>, <Metabolite cpd10516_e at 0x129b9b240>]


In [10]:
from medusa.reconstruct.expand import iterative_gapfill_from_binary_phenotypes

In [11]:
#change so that CO2 can only leave the system
# Add CO2 diffusion
rxn = universal.reactions.rxn05467_c.copy()
model.add_reactions([rxn])
# constrain lower bound of CO2 exchange to be 0 or positive
model.reactions.EX_cpd00011_e.lower_bound = 0
# prevent forward flux through the CO2 diffusion reaction (e -> c)
universal.reactions.rxn30753_c.upper_bound = 0
model.reactions.rxn05467_c.upper_bound = 0

# prevent net production of O2
model.reactions.EX_cpd00007_e.upper_bound = 0


# Make O2-producing reactions in the universal non-reversible
O2_rxns = [r.id for r in universal.reactions if 'cpd00007_c' in [m.id for m in r.metabolites]]
for rxn in O2_rxns:
    r_obj = universal.reactions.get_by_id(rxn)
    if 'cpd00007_c' in [r.id for r in r_obj.reactants]:
        r_obj.lower_bound = 0
    elif 'cpd00007_c' in [r.id for r in r_obj.products]:
        r_obj.upper_bound = 0
        
# repeat for reactions in the model
print('Modifying model rxns...\n\n')
O2_rxns = [r.id for r in model.reactions if 'cpd00007_c' in [m.id for m in r.metabolites]]
for rxn in O2_rxns:
    r_obj = model.reactions.get_by_id(rxn)
    # as long as the reaction was not a transport reaction, prevent it form producing cytosolic O2
    all_compartments = [m.compartment for m in r_obj.metabolites]
    if len(set(all_compartments)) < 2:
        if 'cpd00007_c' in [r.id for r in r_obj.reactants]:
            r_obj.lower_bound = 0
        elif 'cpd00007_c' in [r.id for r in r_obj.products]:
            r_obj.upper_bound = 0

# same for CO2, except it should never be fixed rather than formed
CO2_rxns = [r.id for r in universal.reactions if 'cpd00011_c' in [m.id for m in r.metabolites]]
for rxn in CO2_rxns:
    r_obj = universal.reactions.get_by_id(rxn)
    if 'cpd00011_c' in [r.id for r in r_obj.products]:
        r_obj.lower_bound = 0
    elif 'cpd00011_c' in [r.id for r in r_obj.reactants]:
        r_obj.upper_bound = 0
        
# repeat for reactions in the model
print('Modifying model rxns...\n\n')
CO2_rxns = [r.id for r in model.reactions if 'cpd00011_c' in [m.id for m in r.metabolites]]
for rxn in CO2_rxns:
    r_obj = model.reactions.get_by_id(rxn)
    # as long as the reaction was not a transport reaction, prevent it form producing cytosolic O2
    all_compartments = [m.compartment for m in r_obj.metabolites]
    if len(set(all_compartments)) < 2:
        if 'cpd00011_c' in [r.id for r in r_obj.products]:
            r_obj.lower_bound = 0
        elif 'cpd00011_c' in [r.id for r in r_obj.reactants]:
            r_obj.upper_bound = 0

# make transport reactions with ATP only operate in the ATP-consuming direction
print('modifying directionality for ATP-producing transport reactions, except F(1) ATPase')
atp_rxns = [r.id for r in model.reactions if 'cpd00002_c' in [m.id for m in r.metabolites]]
for rxn in atp_rxns:
    r_obj = model.reactions.get_by_id(rxn)
    # as long as the reaction was not a transport reaction, prevent it form producing cytosolic O2
    all_compartments = [m.compartment for m in r_obj.metabolites]
    if len(set(all_compartments)) > 1:
        if rxn is not 'rxn10042_c':
            if 'cpd00002_c' in [r.id for r in r_obj.reactants]:
                r_obj.lower_bound = 0
            elif 'cpd00002_c' in [r.id for r in r_obj.products]:
                r_obj.upper_bound = 0

atp_rxns = [r.id for r in universal.reactions if 'cpd00002_c' in [m.id for m in r.metabolites]]
for rxn in atp_rxns:
    r_obj = universal.reactions.get_by_id(rxn)
    # as long as the reaction was not a transport reaction, prevent it form producing cytosolic O2
    all_compartments = [m.compartment for m in r_obj.metabolites]
    if len(set(all_compartments)) > 1:
        if rxn is not 'rxn10042_c':
            if 'cpd00002_c' in [r.id for r in r_obj.reactants]:
                r_obj.lower_bound = 0
            elif 'cpd00002_c' in [r.id for r in r_obj.products]:
                r_obj.upper_bound = 0


Ignoring reaction 'rxn05467_c' since it already exists.


Modifying model rxns...


Modifying model rxns...


modifying directionality for ATP-producing transport reactions, except F(1) ATPase


In [12]:
# Add an exchange reaction so the biomass metabolite can leave the system
ex_rxn = Reaction("EX_cpd11416_c")
ex_rxn.lower_bounds = 0
ex_rxn.upper_bounds = 1000
ex_rxn.add_metabolites({model.metabolites.cpd11416_c:-1})
model.add_reactions([ex_rxn])

In [13]:
#sometimes the ensemble will skip a member number during generation, I don't know why. But that is why I generate an ensebmle with a few extra members to work with incase this happens
num_cycles = 100
lower_bound = 0.05
flux_cutoff = 1E-15
ensemble = iterative_gapfill_from_binary_phenotypes(model,universal,media_dicts,num_cycles,\
                                     lower_bound=lower_bound,\
                                     inclusion_threshold=flux_cutoff,\
                                     exchange_reactions=False,\
                                     demand_reactions=False,\
                                     exchange_prefix='EX_')

Constraining lower bound for bio1
starting cycle number 0
starting cycle number 1
starting cycle number 2
starting cycle number 3
starting cycle number 4
starting cycle number 5
starting cycle number 6
starting cycle number 7
starting cycle number 8
starting cycle number 9
starting cycle number 10
starting cycle number 11
starting cycle number 12
starting cycle number 13
starting cycle number 14
starting cycle number 15
starting cycle number 16
starting cycle number 17
starting cycle number 18
starting cycle number 19
starting cycle number 20
starting cycle number 21
starting cycle number 22
starting cycle number 23
starting cycle number 24
starting cycle number 25
starting cycle number 26
starting cycle number 27
starting cycle number 28
starting cycle number 29
starting cycle number 30
starting cycle number 31
starting cycle number 32
starting cycle number 33
starting cycle number 34
starting cycle number 35
starting cycle number 36
starting cycle number 37
starting cycle number 38
s

In [14]:
#save to the proper directory please
save_dir = ("../results/ensembles/psy_ensemble_100_for_analysis.pickle")
ensemble.to_pickle(save_dir)

Below are some cells that were used as sanity checks during development to make sure CO2 was not being consumed and O2 was not being produced.

In [15]:
from medusa.flux_analysis import flux_balance
fluxes = flux_balance.optimize_ensemble(ensemble,return_flux='bio1')
fluxes

,bio1
PST_gapfilled_0,541.882620
PST_gapfilled_1,541.091566
PST_gapfilled_2,557.644253
PST_gapfilled_3,521.844678
PST_gapfilled_4,509.559847
...,...
PST_gapfilled_95,528.775700
PST_gapfilled_96,515.077256
PST_gapfilled_97,525.644025
PST_gapfilled_98,549.996888


In [16]:
ensemble.set_state(ensemble.members[0])
gf_mod1 = ensemble.base_model.copy()

In [17]:
gf_mod1.reactions.EX_cpd00007_e

Reaction identifier,EX_cpd00007_e
Name,Exchange reaction for cpd00007_e
Memory address,0x01281134e0
Stoichiometry,cpd00007_e <-- O2 <--
GPR,
Lower bound,-1000
Upper bound,0


In [18]:
active_exchanges = gf_mod1.optimize().fluxes[[r.id for r in gf_mod1.boundary]][abs(gf_mod1.optimize().fluxes[[r.id for r in gf_mod1.boundary]]) > 1E-10]
active_exchanges = pd.DataFrame(active_exchanges)
active_exchanges['met_name'] = [gf_mod1.metabolites.get_by_id(ind[3:]).name for ind in active_exchanges.index]
active_exchanges.sort_values(by='fluxes')


,fluxes,met_name
EX_cpd01307_e,-1000.000000,D-Lyxitol
EX_cpd00129_e,-1000.000000,L-Proline
EX_cpd00036_e,-1000.000000,Succinate
EX_cpd00246_e,-1000.000000,Inosine
EX_cpd00106_e,-1000.000000,Fumarate
EX_cpd00121_e,-1000.000000,L-Inositol
EX_cpd00132_e,-1000.000000,L-Asparagine
EX_cpd00076_e,-798.762171,Sucrose
EX_cpd00007_e,-675.578066,O2
EX_cpd00130_e,-629.140705,L-Malate


In [19]:
gf_mod1.medium = media_dicts['cpd00129']
for rxn in gf_mod1.boundary:
    if rxn.id not in media_dicts['cpd00129'].keys():
        rxn.lower_bound = 0
gf_mod1.reactions.EX_cpd00129_e.lower_bound = -1
print(gf_mod1.optimize())
pfba_solution = cobra.flux_analysis.pfba(gf_mod1)

<Solution 0.266 at 0x11081b3c8>


In [20]:
#this is also included for debugging purposes. Some exchange reactions are in the universal reaction bag that were being added into the model, this was creating an issue 
#where metabolites were coming in when they should not be because there is no evidence for their use. 
##another possible solution would be to keep these reactions in but add a very large penelty for their inclusion into the ensemble
for rxn in universal.reactions:
    if "EX_" in rxn.id:
        universal.remove_reactions([rxn], remove_orphans = True)
    else:
        pass

In [21]:
active_exchanges = pfba_solution.fluxes[[r.id for r in gf_mod1.boundary]][abs(pfba_solution.fluxes[[r.id for r in gf_mod1.boundary]]) > 1E-10]
active_exchanges = pd.DataFrame(active_exchanges)
active_exchanges['met_name'] = [gf_mod1.metabolites.get_by_id(ind[3:]).name for ind in active_exchanges.index]
active_exchanges.sort_values(by='fluxes')

,fluxes,met_name
EX_cpd00129_e,-1.000000,L-Proline
EX_cpd00007_e,-0.355377,O2
EX_cpd00048_e,-0.007005,Sulfate
EX_cpd10515_e,-0.002471,Fe2+
EX_cpd10516_e,-0.000824,fe3
EX_cpd00058_e,-0.000824,Cu2+
EX_cpd00030_e,-0.000824,Mn2+
EX_cpd00034_e,-0.000824,Zn2+
EX_cpd00063_e,-0.000824,Ca2+
EX_cpd00149_e,-0.000824,Co2+
